In [84]:
# all import request for scrapping activity
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import validators
from itertools import cycle

In [197]:
search_terms_dict = {
    # "Technology": "Technology OR IT OR Machine Learning OR Network Security",
    # "Healthcare": "Healthcare OR Pharmaceuticals OR Clinical Trials",
    # "Finance": "Finance OR Banking OR Retail Banking",
    # "Education": "Education OR Online Learning OR Vocational Training",
    # "Manufacturing": "Manufacturing OR Aerospace OR Electronics OR Mechanical",
    # "Energy": "Energy OR Renewable OR Nuclear Power OR Hydroelectric Power OR Biofuels",
    # "Retail": "Retail OR E-commerce OR Online Marketplaces OR Luxury Goods OR Supermarkets",
    # "Construction": "Construction OR Real Estate OR Architecture OR Interior Design OR Property Management",
    # "Transportation": "Transportation OR Logistics OR Air Freight OR Railroads OR Maritime OR Courier",
    # "Hospitality": "Hospitality OR Tourism OR Culinary Arts OR Theme Parks OR Cruise Lines",
    "Professional Services": "Professional Services OR Audit OR Accounting OR Executive Search OR Digital Marketing",
    "Creative Media": "Creative Media OR Video Production OR Digital Content OR Graphic Design OR Music Industry",
    "Telecommunications": "Telecommunications OR Broadband OR Satellite Communications OR ISPs OR VoIP",
    "Agriculture": "Agriculture OR Forestry OR Organic Farming OR Aquaculture OR Agrochemicals",
    "Public Sector": "Public Sector OR Non-Profit OR Civic Organization OR International Development OR Environmental",
    "Science Research": "Science Research OR Nanotechnology OR Genomics OR Environmental Science OR Astrophysics",
    "Engineering": "Engineering OR Robotics OR Biomedical OR Environmental OR Mechanical OR Software",
    "Emerging Industries": "Emerging Industries OR VR OR AR OR CleanTech OR Space Exploration OR FinTech"
}


In [85]:
# User agents and headers for the request
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0",
    "Mozilla/5.0 (Linux; Android 10; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.210 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko",
    "Mozilla/5.0 (iPad; CPU OS 13_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:78.0) Gecko/20100101 Firefox/78.0"
]


user_agents_cycle = cycle(user_agents)

In [86]:
def error_handler(target_url, query_params, error_response):
    
    retry_after = int(error_response.headers.get('Retry-After', 30))  # Default to 60 seconds if header is missing
    print(f"Rate limit hit. Retrying after {retry_after} seconds.")
    time.sleep(retry_after)
    
    while True:
        response = requests.get(target_url, params=query_params, headers={"User-Agent": next(user_agents_cycle)})
        if response.status_code == 200:
        # print("Request was successful!")
            time.sleep(10)
            return BeautifulSoup(response.text,'html.parser')
        elif response.status_code == 429:
            time.sleep(30)
        else:
            break

# request send function
def send_request(target_url, query_params):
    response = requests.get(target_url, params=query_params, headers={"User-Agent": next(user_agents_cycle)})
    if response.status_code == 200:
        return BeautifulSoup(response.text,'html.parser')
    else:
        res = error_handler(target_url,query_params, response)
        if res is None:
                   pass
        else:
            return res
    

In [87]:
#converting string typed count to integer
def job_count_converter(soup):
    count = soup.find(class_='results-context-header__job-count').text
    return  int(re.sub(r'[^0-9]', '', count))  # Remove anything that's not a digit

In [88]:
def crab_job_links_using_keyword(keyword:str):
    # Main body of the URL
    job_search_url = "https://www.linkedin.com/jobs/search"

    # Query parameters as a dictionary, updated with the new keyword
    query_params = {
    "keywords": keyword,
    "location": "Canada",
    "geoId": "",
    "f_TPR": "r604800",
    "position": "1",
    "pageNum": "0",
    "start": "0",
    "sortBy": "DD"
}
    jobs_crabbed  = 0
    jobs_posting_link = [] 
    soup = send_request(job_search_url, query_params)
    if soup is None:
        print("Request failed. Exiting. at the first request")
    else:
        number = job_count_converter(soup)
        counter = 1000
        if number <= 1000:
            counter = number
        while jobs_crabbed <= counter:
            if not jobs_crabbed == 0:
                query_params["start"] = jobs_crabbed
                print(jobs_crabbed)
                soup = send_request(job_search_url, query_params)
                time.sleep(5)
            job_list_html = soup.find('ul', class_="jobs-search__results-list").find_all('li')
            jobs_crabbed += len(job_list_html)
            for job in job_list_html:
                try:
                    job_link = job.find('a', class_="base-card__full-link")['href']
                except:
                    job_link = None
                jobs_posting_link.append(job_link)
        print(f"{keyword}: {len(jobs_posting_link)}")
        return jobs_posting_link

# create a pandas series using list job list posting and save.
def saving_file(jobs_posting_link, keyword:str):
    series = pd.Series(jobs_posting_link)
    series.to_csv(f'{keyword}.csv', index=False)

In [80]:
# import requests
# from bs4 import BeautifulSoup
# import math
# import pandas as pd
# l=[]
# o={}
# k=[]
# headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
# target_url='https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Python%20%28Programming%20Language%29&location=Las%20Vegas%2C%20Nevada%2C%20United%20States&geoId=100293800&currentJobId=3415227738&start={}'
# for i in range(0,math.ceil(117/25)):

#     res = requests.get(target_url.format(i))
#     soup=BeautifulSoup(res.text,'html.parser')
#     alljobs_on_this_page=soup.find_all("li")
#     print(len(alljobs_on_this_page))
#     for x in range(0,len(alljobs_on_this_page)):
#         jobid = alljobs_on_this_page[x].find("div",{"class":"base-card"}).get('data-entity-urn').split(":")[3]
#         l.append(jobid)

# target_url='https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
# for j in range(0,len(l)):

#     resp = requests.get(target_url.format(l[j]))
#     soup=BeautifulSoup(resp.text,'html.parser')

#     try:
#         o["company"]=soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
#     except:
#         o["company"]=None

#     try:
#         o["job-title"]=soup.find("div",{"class":"top-card-layout__entity-info"}).find("a").text.strip()
#     except:
#         o["job-title"]=None

#     try:
#         o["level"]=soup.find("ul",{"class":"description__job-criteria-list"}).find("li").text.replace("Seniority level","").strip()
#     except:
#         o["level"]=None



#     k.append(o)
#     o={}

# df = pd.DataFrame(k)
# df.to_csv('linkedinjobs.csv', index=False, encoding='utf-8')
# print(k)

In [ ]:
for keyword in search_terms_dict:
    print(f"Starting with {search_terms_dict[keyword]} jobs")
    jobs_posting_link = crab_job_links_using_keyword(search_terms_dict[keyword])
    saving_file(jobs_posting_link, keyword)
    print(f"Done with {keyword} jobs")


In [ ]:
# {
#     "Job role": "soup.find("h1",{"class":"topcard__title"}).text.strip",
#     "Company": "soup.find("a",{"class":"topcard__org-name-link topcard__flavor--black-link"}).text.strip",
#     "Location": "soup.find("span",{"class":"topcard__flavor topcard__flavor--bullet"}).text.strip",
#     "Experience Level": "soup.find(lambda tag: tag.name == 'h3' and 'Seniority level' in tag.text).find_next_sibling("span").text.strip",
#     "Posted time ago": "soup.find("span",{"class":"posted-time-ago__text"}).text.strip",
#     "Job Type": soup.find(lambda tag: tag.name == 'h3' and 'Employment type' in tag.text).find_next_sibling("span").text.strip,
#     "Job Function category": soup.find(lambda tag: tag.name == 'h3' and 'Job function' in tag.text).find_next_sibling("span").text.strip,
#     "Industry Category": "soup.find(lambda tag: tag.name == 'h3' and 'Industries' in tag.text).find_next_sibling("span").text.strip",
# }

In [181]:
def try_except(function):
    try:
        return function()
    except:
        return None

def get_job_details(soup):
    return {
        "Job role": try_except(lambda: soup.find("h1",{"class":"topcard__title"}).text.strip()),
        "Company": try_except(lambda: soup.find("a",{"class":"topcard__org-name-link topcard__flavor--black-link"}).text.strip()),
        "Location": try_except(lambda: soup.find("span",{"class":"topcard__flavor topcard__flavor--bullet"}).text.strip()),
        "Experience Level": try_except(lambda: soup.find(lambda tag: tag.name == 'h3' and 'Seniority level' in tag.text).find_next_sibling("span").text.strip()),
        "Posted time ago": try_except(lambda: soup.find("span",{"class":"posted-time-ago__text"}).text.strip()),
        "Job Type": try_except(lambda: soup.find(lambda tag: tag.name == 'h3' and 'Employment type' in tag.text).find_next_sibling("span").text.strip()),
        "Job Function category": try_except(lambda: soup.find(lambda tag: tag.name == 'h3' and 'Job function' in tag.text).find_next_sibling("span").text.strip()),
        "Industry Category": try_except(lambda: soup.find(lambda tag: tag.name == 'h3' and 'Industries' in tag.text).find_next_sibling("span").text.strip()),
        "Job Description": try_except(lambda: soup.find("div",{"class":"show-more-less-html__markup"}).text.strip())
    }

In [183]:
def filtering_series(series):
    def is_valid_url(url):
        if pd.isna(url):  # Check if the value is NaN
            return False  # Consider NaN as invalid
        return validators.url(url)

    # Apply the function to check if each URL is valid, and filter the Series
    valid_urls_mask = series.apply(is_valid_url)

    # Use the mask to filter the Series and drop NaN values
    return series[valid_urls_mask].dropna()


In [198]:
for keyword in search_terms_dict:
    print(f"Starting with {search_terms_dict[keyword]} jobs")
    df = pd.read_csv(f'{keyword}.csv')
    series = df["0"]
    series = filtering_series(series)
    df_posting_data = pd.DataFrame(columns=[
    "Job role",
    "Company",
    "Location",
    "Experience Level",
    "Posted time ago",
    "Job Type",
    "Job Function category",
    "Industry Category",
    "Job Description"
])
    for i in series.index:
        soup = send_request(series[i], {})
        time.sleep(4)
        data = get_job_details(soup)
        for key in data:
            df_posting_data.loc[i, key] = data[key]
        print(f"Done with {i} jobs")
    
    df_posting_data.to_csv(f'dataframe/{keyword}.csv', index=False, encoding='utf-8')
    print(f"Done with {search_terms_dict[keyword]} jobs")

Starting with Healthcare OR Pharmaceuticals OR Clinical Trials jobs
Done with 0 jobs
Done with 1 jobs
Done with 2 jobs
Done with 3 jobs
Done with 4 jobs
Rate limit hit. Retrying after 30 seconds.
Done with 5 jobs
Done with 6 jobs
Done with 7 jobs
Done with 8 jobs
Rate limit hit. Retrying after 30 seconds.
Done with 9 jobs
Done with 10 jobs
Done with 11 jobs
Done with 12 jobs
Done with 13 jobs
Done with 14 jobs
Done with 15 jobs
Done with 16 jobs
Done with 17 jobs
Done with 18 jobs
Done with 19 jobs
Done with 20 jobs
Done with 21 jobs
Done with 22 jobs
Done with 23 jobs
Done with 24 jobs
Done with 25 jobs
Done with 26 jobs
Done with 27 jobs
Rate limit hit. Retrying after 30 seconds.
Done with 28 jobs
Done with 29 jobs
Done with 30 jobs
Done with 31 jobs
Done with 32 jobs
Done with 33 jobs
Done with 34 jobs
Done with 35 jobs
Done with 36 jobs
Done with 37 jobs
Done with 38 jobs
Done with 39 jobs
Done with 40 jobs
Rate limit hit. Retrying after 30 seconds.
Done with 41 jobs
Done with 42 j

KeyboardInterrupt: 